In [1]:
import numpy as np
import pandas as pd
import pickle
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU, Embedding

Using TensorFlow backend.


In [30]:
hdw_vec = pd.read_csv('../data/cat_train_HDW_MMAO15.csv',nrows=10000)

In [31]:
hdw_vec.head()

,datetime,tot_min,A01-1_Headway_0,A01-1_Headway_1,A01-1_Headway_2,A01-2_Headway_0,A01-2_Headway_1,A01-2_Headway_2,A02-1_Headway_0,A02-1_Headway_1,...,N02-2_Headway_2,N03-1_Headway_0,N03-1_Headway_1,N03-1_Headway_2,N03-2_Headway_0,N03-2_Headway_1,N03-2_Headway_2,N04-1_Headway_0,N04-1_Headway_1,N04-1_Headway_2
0,2015-3-1 7:59,85439.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015-3-1 8:0,85440.0,1,0,0,1,0,0,1,0,...,0,1,0,0,0,1,0,1,0,0
2,2015-3-1 8:1,85441.0,0,1,0,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
3,2015-3-1 8:2,85442.0,0,0,1,0,1,0,0,1,...,0,1,0,0,0,1,0,1,0,0
4,2015-3-1 8:3,85443.0,0,1,0,0,1,0,1,0,...,0,0,0,1,1,0,0,0,0,1


In [32]:
t = hdw_vec['tot_min'].tolist()

In [33]:
xh_tt = []
yh_tt = []
for i in range(30,len(hdw_vec)-30):
    if t[i] - t[i-30] == 30. and t[i+30] - t[i] == 30.:
        x_vec = []
        x_vec.append(hdw_vec.loc[i-30][2:].tolist())
        x_vec.append(hdw_vec.loc[i-15][2:].tolist())
        x_vec.append(hdw_vec.loc[i][2:].tolist())
        xh_tt.append(x_vec)
        y_vec = []
        y_vec.append(hdw_vec.loc[i+15][2:].tolist())
        y_vec.append(hdw_vec.loc[i+30][2:].tolist())
        yh_tt.append(y_vec)

In [34]:
xh2 = np.asarray(xh_tt)

In [35]:
xh2.shape

(9340, 3, 513)

In [32]:
xh_tt = []
yh_tt = []
tt = []
chunk = 10000
k = -1
for hdw_vec in pd.read_csv('../data/cat_train_HDW_MMAO15.csv',chunksize=chunk):
    print('iter ',k+1)
    k += 1
    t = hdw_vec['tot_min'].tolist()
    for i in range(30+k*chunk,len(hdw_vec)-30+k*chunk):
        if t[i-k*chunk] - t[i-30-k*chunk] == 30.  \
                      and t[i+30-k*chunk] - t[i-k*chunk] == 30.:
            x_vec = []
            x_vec.append(hdw_vec.loc[i-30][2:].tolist())
            x_vec.append(hdw_vec.loc[i-15][2:].tolist())
            x_vec.append(hdw_vec.loc[i][2:].tolist())
            xh_tt.append(x_vec)
            y_vec = []
            y_vec.append(hdw_vec.loc[i+15][2:].tolist())
            y_vec.append(hdw_vec.loc[i+30][2:].tolist())
            yh_tt.append(y_vec)
            tt.append(t[i+30-k*chunk])
    if len(xh_tt) > 60000:
        break

iter  0
iter  1
iter  2
iter  3
iter  4
iter  5
iter  6


In [4]:
xh2 = np.asarray(xh_tt)
yh2 = np.asarray(yh_tt)
print(xh2.shape)
print(yh2.shape)

(65020, 3, 513)
(65020, 2, 513)


In [5]:
del xh_tt
del yh_tt

In [5]:
import sklearn.model_selection

In [6]:
max_encoder_seq_length = 3
max_decoder_seq_length = 2

In [7]:
num_encoder_tokens = 513
num_decoder_tokens = 513

In [8]:
encoder_input_data = xh2
decoder_target_data = yh2

In [11]:
decoder_input_data = []
decoder_start = [0.5] * 513
for i in range(len(yh2)):
    yi_seq = []
    yi_seq.append(decoder_start)
    yi_seq.append(yh2[i][0])
    decoder_input_data.append(yi_seq)
decoder_input_data = np.asarray(decoder_input_data, dtype = float)

In [13]:
del xh2, yh2

In [14]:
decoder_input_data.shape

(65020, 2, 513)

In [16]:
batch_size = 64
epochs = 2
latent_dim = 256

In [17]:
# after the model of https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

# encoder architecture
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# retain final state to condition decoder; discard encoder output seq
encoder_states = [state_h, state_c]

# decoder architecture
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='sigmoid')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
# define model ... inputs are to encoder and decoder, respectively; output from decoder only
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [19]:
# SGD with Adam optimizer, crossentropy loss
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs)

Epoch 1/2
65020/65020 [==============================] - 136s - loss: 896.1761 - acc: 4.0757e-04   
Epoch 2/2
65020/65020 [==============================] - 123s - loss: 888.0690 - acc: 8.3820e-04   


In [21]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [23]:
def decode_sequence(input_seq):
    pred_seq = []
    # Encode the input into state to pass to decoder
    states_value = encoder_model.predict(input_seq)

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    for i in range(num_decoder_tokens):
        target_seq[0, 0, i] = 0.5

    for i in range(2):
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        pred_seq.append(output_tokens.reshape(num_decoder_tokens).tolist())
        # Update the target sequence
        target_seq = output_tokens
        # Update states
        states_value = [h, c]

    return pred_seq

In [25]:
pred_seq=[]
for seq_index in range(len(encoder_input_data)-1):
    input_seq = encoder_input_data[seq_index:seq_index+1]
    pred_seq.append(decode_sequence(input_seq))

In [26]:
pred_s = np.asarray(pred_seq)
pred_s.shape

(65019, 2, 513)

In [27]:
def tokenz(loc_vec):  # three element probability output for platform HDW state at +15 min or +30 min
    vec_max = max(loc_vec)
    if vec_max < 0.33:
        return 0
    else:
        return np.argmax(loc_vec) + 1
# token def'n:  0 = [0,0,0] = nan, 1 = [1,0,0], 2 = [0,1,0], 3 = [0,0,1]

In [28]:
pred_tok_15 = []
pred_tok_30 = []
for i in range(len(pred_seq)):
    vec_15 = []
    vec_30 = []
    for j in range(171):
        loc_15 = pred_seq[i][0][3*j:3*(j+1)]
        loc_30 = pred_seq[i][1][3*j:3*(j+1)]
        vec_15.append(tokenz(loc_15))
        vec_30.append(tokenz(loc_30))
    pred_tok_15.append(vec_15)
    pred_tok_30.append(vec_30)

In [29]:
pred_t15 = np.asarray(pred_tok_15)
pred_t15.shape

(65019, 171)

In [35]:
f_name = '../data/pred_tok_15_lrg.pkl'
f_obj = open(f_name,'wb')
pickle.dump(pred_tok_15,f_obj)
f_obj.close()
f_name = '../data/pred_tok_30_lrg.pkl'
f_obj = open(f_name,'wb')
pickle.dump(pred_tok_30,f_obj)
f_obj.close()

In [34]:
f_name = '../data/tot_time_30_lrg.pkl'
f_obj = open(f_name,'wb')
pickle.dump(tt,f_obj)
f_obj.close()